In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Prepare people data frame
df_people = pd.read_csv('Data/demog.csv')
print(df_people.shape)
# Keep only people for which we have age
df_people = df_people[~pd.isna(df_people.age)]
print(df_people.shape)
# Target teenagers
df_people['target_age'] = df_people.age >= 50
#df_people = df_people[['userid', 'target_age', 'age']]
df_people.userid = df_people.userid.astype('category')
df_people.head()

(4282857, 15)
(1573876, 15)


,userid,gender,birthday,age,relationship_status,interested_in,mf_relationship,mf_dating,mf_random,mf_friendship,mf_whatever,mf_networking,locale,network_size,timezone,target_age
6,631f821671a94f991413e58c2b393b29,1.0,1982-08-09,30.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en_US,NaN,NaN,False
7,bdeb2156021c5da9237f5d2667ab590a,1.0,1983-07-26,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en_US,681.0,-5.0,False
12,d9fb05627d4659484703e5827690b9f9,0.0,1985-01-21,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en_US,586.0,-4.0,False
23,adc399722fccda28049cb26c5a795b16,1.0,1983-12-19,29.0,2.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,en_US,300.0,-5.0,False
29,18f92f9c1f86b5cd692e8fcc19721b1d,1.0,1985-02-19,27.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,en_US,532.0,-7.0,False


In [2]:
# Prepare likes data frame
df_likes = pd.read_csv('Data/likes.csv', header=None, names=['userid', 'like_id'])
df_likes.userid = df_likes.userid.astype('category')
print(df_likes.shape)

(55775701, 2)


In [3]:
# Prune to keep only pages with more than 1,000 users and users with at least one like
change = True
while change:
    # Keep only likes of people for which we have information
    print("Likes before and after")
    val = df_likes.shape[0]
    print(df_likes.shape)
    df_likes = df_likes[df_likes.userid.isin(df_people.userid)]
    print(df_likes.shape)
    change = val != df_likes.shape[0]

    # Keep only people with at least one like
    print("People before and after")
    val = df_people.shape[0]
    print(df_people.shape)
    df_people = df_people[df_people.userid.isin(df_likes.userid.unique())]
    print(df_people.shape)
    change = (val != df_people.shape[0]) | change
    # Keep only pages with at least 1,000 likes
    print("Pages before and after")
    counts = df_likes.like_id.value_counts()
    val = len(counts)
    high_counts = counts[counts >= 1000]
    change = (val != len(high_counts)) | change
    print(len(counts))
    df_likes = df_likes[df_likes.like_id.isin(high_counts.index.values)]
    print(len(high_counts))

Likes before and after
(55775701, 2)
(37686388, 2)
People before and after
(1573876, 16)
(588610, 16)
Pages before and after
15914
10822
Likes before and after
(33873012, 2)
(33873012, 2)
People before and after
(588610, 16)
(587745, 16)
Pages before and after
10822
10822
Likes before and after
(33873012, 2)
(33873012, 2)
People before and after
(587745, 16)
(587745, 16)
Pages before and after
10822
10822


In [4]:
# Get labels
labels = df_people.set_index('userid').loc[sorted(df_likes.userid.unique())].target_age.values
ages = df_people.set_index('userid').loc[sorted(df_likes.userid.unique())].age.values
# Build sparse matrix
users_c = np.array(sorted(df_likes.userid.unique()))
pages_c = np.array(sorted(df_likes.like_id.unique()))
row = pd.Categorical(df_likes.userid, categories=users_c, ordered=True).codes
col = pd.Categorical(df_likes.like_id, categories=pages_c, ordered=True).codes
sparse_mat = csr_matrix((np.ones(df_likes.shape[0]), (row, col)), shape=(users_c.size, pages_c.size))
sparse_mat

<587745x10822 sparse matrix of type '<class 'numpy.float64'>'
	with 33873012 stored elements in Compressed Sparse Row format>

In [5]:
# Build machine learning model
X_train, X_test, y_train, y_test, age_train, age_test = train_test_split(sparse_mat, labels, ages,
                                                                         test_size=0.3, random_state=42)

model = LogisticRegression(random_state=42, solver='lbfgs').fit(X_train, y_train)
fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:, 1])
metrics.auc(fpr, tpr)

C:\Users\ferlo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8574013931369411

In [6]:
# Load names of facebook pages
df_names = pd.read_csv('Data/fb_like.csv', encoding='latin-1', error_bad_lines=False, warn_bad_lines=False)
print(df_names.shape)
df_names.head()
df_names = df_names[df_names.like_id.isin(pages_c)]
print(df_names.shape)
df_names = df_names.set_index('like_id')
df_names.head()

(19041200, 2)
(10821, 2)


,name
like_id,
9254817079,photography
103144503058854,Poetry
104046862964234,Writing
105667052799444,Piano
106059522759137,English


In [7]:
# Get threshold
probs = model.predict_proba(X_test)[:, 1]
threshold = np.percentile(probs, 99)
print("To target top 1%, target individuals with more than: ", threshold)
# Keep only people with probabilities above threshold
top_x = X_test[probs > threshold, :]
top_y = y_test[probs > threshold]
top_age = age_test[probs > threshold]
# Take a look at average number of likes
print("AVG number of likes of targeted people:", top_x.sum(axis=1).mean())
print("Median number of likes of targeted people:",np.median(np.array(top_x.sum(axis=1))))
# Keep interesting errors
top_x = top_x[top_age >= 30]
top_y = top_y[top_age >= 30]
top_age = top_age[top_age >= 30]
top_x = top_x[top_age <= 40]
top_y = top_y[top_age <= 40]
top_age = top_age[top_age <= 40]
print("Misclassified individuals between 30 and 40 years:", top_x.shape[0])
# Take a look at probabilities
print("Probabilities of misclassified individuals:")
print(model.predict_proba(top_x)[:, 1])
print("Number of likes of misclassified individuals:")
print(top_x.sum(axis=1).reshape(1, -1))
order = np.array(top_x.sum(axis=1)).reshape(-1).argsort()[::-1]
top_x = top_x[order]
top_y = top_y[order]
top_age = top_age[order]

To target top 1%, target individuals with more than:  0.41116640934620974
AVG number of likes of targeted people: 73.29024943310658
Median number of likes of targeted people: 41.0
Misclassified individuals between 30 and 40 years: 165
Probabilities of misclassified individuals:
[0.84565248 0.74256652 0.78119628 0.72119612 0.41385715 0.68499266
 0.45898055 0.98829092 0.80072783 0.43934153 0.81048513 0.73170057
 0.58773163 0.41981386 0.74491212 0.55494788 0.47974449 0.50910381
 0.47437852 0.58547422 0.53424198 0.49039252 0.69717416 0.8333557
 0.75581428 0.44579571 0.56518143 0.92140531 0.41324849 0.42370999
 0.461326   0.48154155 0.74343522 0.57035595 0.84301434 0.51596466
 0.57772989 0.60031359 0.42048403 0.41358187 0.44502888 0.62397304
 0.65683409 0.86094985 0.68844824 0.41694971 0.99615006 0.77394067
 0.5744485  0.76198515 0.5085968  0.5547998  0.76078767 0.51821855
 0.53853298 0.75322157 0.74039866 0.88581494 0.49459344 0.98338893
 0.46801608 0.6121224  0.98678264 0.81081969 0.46479

In [8]:
print("Number of likes of Percentile 67:")
np.percentile(np.array(X_test[probs > threshold, :].sum(axis=1)).flatten(), 67)

Number of likes of Percentile 67:


64.0

In [9]:
import shap

# Candidates 12, 3, 15, 21
# Candidates 32, 74, 88, 39, 41, 46

def make_decision(data):
    return (model.predict_proba(data)[:, 1] > threshold).astype(float)

#for i in range(30, 99):
# Choose arbitrary individual for the analysis
i = 37
print("----------------------------------------------")
print("Misclassified Individual Index: ", i)
top_x_guy = top_x[np.full(5, i), :]
print("Number of likes:", top_x_guy[0, :].sum())
print("Age:", top_age[i])
print("Probability of being old:", model.predict_proba(top_x_guy)[0, 1])
np.random.seed(0)
default = csr_matrix(np.zeros(X_test.shape[1]))
shap_explainer = shap.KernelExplainer(make_decision, default)
shap_values = shap_explainer.shap_values(top_x_guy, nsamples=4100, l1_reg='aic')

print("TOP 3 SHAP VALUES, 5 runs")
# Top 3 predictive features are different every time. 
for i in range(shap_values.shape[0]):
    i_vals = (-shap_values[i, :]).argsort()
    print(df_names.loc[pages_c[i_vals[:3]]].name)
    print(shap_values[i, i_vals[:3]])

----------------------------------------------
Misclassified Individual Index:  37
Number of likes: 64.0
Age: 34.0
Probability of being old: 0.41962539207798977


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.85s/it]


TOP 3 SHAP VALUES, 5 runs
like_id
7284978791       ELVIS PRESLEY
182736663312    Paul McCartney
21931600316         Neil Young
Name: name, dtype: object
[0.15816004 0.14952443 0.14929379]
like_id
182736663312    Paul McCartney
21931600316         Neil Young
55555550744     Brain Pickings
Name: name, dtype: object
[0.15839785 0.1552504  0.14245585]
like_id
182736663312    Paul McCartney
21931600316         Neil Young
55555550744     Brain Pickings
Name: name, dtype: object
[0.16662192 0.15264056 0.14765539]
like_id
182736663312    Paul McCartney
38026784643      Leonard Cohen
7284978791       ELVIS PRESLEY
Name: name, dtype: object
[0.15766498 0.15552602 0.14593034]
like_id
182736663312       Paul McCartney
7401700249      Bruce Springsteen
7284978791          ELVIS PRESLEY
Name: name, dtype: object
[0.17209662 0.14778275 0.1358186 ]


In [10]:
# See how long it takes for our method to find one explanation
import explainer
import time

def scoring_function(data):
    return model.predict_proba(data)[:, 1]

# Get evidence-based explanations
explain = explainer.Explainer(scoring_function, default, prune=False)
t0 = time.time()
explanations = explain.explain(top_x_guy[0, :], thresholds=float(threshold), max_ite=50)
t1 = time.time()
print(f"Time to find counterfactual explanations for individual above")
print(t1-t0)
print("Explanations found")
print(len(explanations[0]))
print("Length of explanation")
print(len(explanations[0][0]))
print("First explanations")
print(df_names.loc[pages_c[explanations[0][0]]])
print(df_names.loc[pages_c[explanations[0][1]]])
print(df_names.loc[pages_c[explanations[0][2]]])
print(df_names.loc[pages_c[explanations[0][3]]])
print(df_names.loc[pages_c[explanations[0][4]]])

Time to find counterfactual explanations for individual above
0.07978582382202148
Explanations found
33
Length of explanation
1
First explanations
                        name
like_id                     
182736663312  Paul McCartney
                     name
like_id                  
7284978791  ELVIS PRESLEY
                   name
like_id                
21931600316  Neil Young
                      name
like_id                   
38026784643  Leonard Cohen
                       name
like_id                    
55555550744  Brain Pickings


In [11]:
from functools import reduce

sample_size = 500
np.random.seed(0)
# Get sample of targeted observations to explain
X_targeted = X_test[probs > threshold, :]
X_sample = X_targeted[np.random.choice(X_targeted.shape[0], sample_size, replace=False), :]
X_to_explain = X_sample[np.arange(X_sample.shape[0]).repeat(5), :]
# Obtain their SHAP values (commented because it takes too long)
#shap_values = shap_explainer.shap_values(X_to_explain, nsamples=4100, l1_reg='aic')
#np.save("Data/back_up_shap_values.npy", shap_values)
shap_values = np.load("Data/back_up_shap_values.npy")
# Find top 5 most informative features
most_informative_f = np.argsort(shap_values, axis=1)[:, -3:]
# Compute number of matches
sample_matches = list()
for i in range(sample_size):
    matches = reduce(np.intersect1d, most_informative_f[(i*5):((i+1)*5), :]).size
    #matches = np.unique(most_informative_f[(i*5):((i+1)*5), :]).size
    sample_matches.append(matches)
    
print(sample_matches)

[1, 3, 3, 1, 3, 2, 2, 2, 0, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2, 1, 2, 3, 2, 3, 3, 2, 3, 0, 3, 3, 1, 1, 1, 0, 0, 3, 1, 1, 1, 2, 3, 3, 2, 2, 1, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 0, 1, 3, 2, 3, 1, 3, 3, 3, 1, 2, 2, 1, 2, 2, 2, 3, 3, 2, 1, 3, 2, 3, 1, 3, 1, 3, 3, 2, 0, 3, 3, 3, 2, 1, 3, 0, 2, 1, 3, 2, 2, 3, 3, 2, 0, 3, 3, 3, 2, 0, 2, 2, 1, 3, 2, 2, 3, 0, 0, 3, 3, 1, 3, 1, 2, 2, 2, 3, 1, 1, 2, 1, 3, 3, 2, 0, 2, 3, 1, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 2, 1, 0, 2, 1, 1, 3, 3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 1, 3, 3, 2, 2, 1, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 0, 0, 3, 3, 2, 1, 3, 1, 2, 1, 2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 1, 3, 3, 1, 2, 2, 3, 1, 1, 2, 2, 0, 3, 1, 1, 2, 3, 2, 3, 3, 2, 2, 0, 3, 2, 3, 1, 3, 3, 2, 3, 2, 2, 1, 3, 2, 3, 3, 1, 1, 1, 1, 0, 2, 1, 3, 0, 3, 0, 3, 1, 2, 3, 2, 1, 1, 3, 3, 2, 1, 3, 3, 2, 3, 0, 2, 2, 2, 3, 3, 3, 3, 3, 0, 2, 1, 2, 1, 3, 2, 2, 3, 0, 3, 2, 0, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 3, 0, 3, 2, 2, 1, 1, 0, 3, 2, 1, 2, 3, 1, 1, 2, 2, 2, 3, 1, 3, 2, 1, 1, 2, 1, 1, 

In [12]:
likes_count = np.array(X_sample.sum(axis=1)).flatten()
cut_points = list()
for i in range(1, 5):
    cut_points.append(np.percentile(likes_count, i*20))
cut_points.append(np.inf)

min_likes = 0
sample_matches = np.array(sample_matches) 
for cp in cut_points:
    print("Upper limit: ", cp)
    selected_obs = (likes_count >= min_likes) & (likes_count < cp)
    print(sample_matches[selected_obs].mean())
    min_likes = cp

Upper limit:  18.0
2.7065217391304346
Upper limit:  34.0
2.2547169811320753
Upper limit:  58.39999999999998
2.1372549019607843
Upper limit:  99.0
1.7142857142857142
Upper limit:  inf
1.1470588235294117


In [13]:
# Get counterfactual explanations
explainer.Explainer(scoring_function, default, prune=False)

t0 = time.time()
sample_e = explain.explain(X_sample, thresholds=float(threshold), max_ite=50, stop_at_first=True)
t1 = time.time()
print("Time to find first explanation for 500 instances")
print(t1-t0)

sample_lens = np.array([len(e[0]) for e in sample_e])
min_likes = 0
for cp in cut_points:
    print("Upper limit: ", cp)
    selected_obs = (likes_count >= min_likes) & (likes_count < cp)
    print(sample_lens[selected_obs].mean())
    min_likes = cp

Time to find first explanation for 500 instances
14.732604742050171
Upper limit:  18.0
1.173913043478261
Upper limit:  34.0
2.0
Upper limit:  58.39999999999998
2.303921568627451
Upper limit:  99.0
3.6020408163265305
Upper limit:  inf
4.901960784313726


In [14]:
# Check out real quantiles
likes_count = X_targeted.sum(axis=1)

print(np.percentile(likes_count, 20))
print(np.percentile(likes_count, 40))
print(np.percentile(likes_count, 60))
print(np.percentile(likes_count, 80))

19.0
33.0
53.0
94.0


In [15]:
from scipy.stats import norm

# Define Cost Function
likes_per_page = np.array(sparse_mat.sum(axis=0)).reshape(-1)
costs = norm.cdf((np.mean(likes_per_page) - likes_per_page) / np.std(likes_per_page))
costs += np.full(X_train.shape[1], 0.1)

def likes_cost_func(original_obs, new_obs):
    return (original_obs - new_obs).dot(costs)

# Method to print explanation details
def print_explanation(e):
    for p in e:
        print(df_names.loc[pages_c[p]].values[0], "| Likes :", int(likes_per_page[p]))

#  Find explanations with/without costs
ixs = [11, 38, 108, 413]
t = float(threshold)
for i in ixs:#range(138, 500):
    # With cost
    cost_e = explain.explain(X_sample[i, :], thresholds=t, max_ite=300, stop_at_first=True,
                             cost_func=likes_cost_func)[0]
    cost_e = cost_e[0] if len(cost_e) > 0 else []
    # No cost
    no_cost_e = explain.explain(X_sample[i, :], thresholds=t, max_ite=300, stop_at_first=True)[0][0]
    print("ID: ", i)
    print("Explanation WITHOUT costs")
    print_explanation(no_cost_e)
    print("---")
    print("Explanation WITH costs")
    print_explanation(cost_e)
    print("*************************************************")

ID:  11
Explanation WITHOUT costs
It's a Wonderful Life | Likes : 1181
JESUS IS LORD!!!!!!!!!!!!!!!!!!!!!!!!!!! if you know this is true press like. :) | Likes : 1291
---
Explanation WITH costs
Reading | Likes : 47288
American Idol | Likes : 15792
Classical | Likes : 8632
*************************************************
ID:  38
Explanation WITHOUT costs
The Hollywood Gossip | Likes : 1353
Let's remember those who have passed. Press Like if you've lost a loved one... | Likes : 2248
---
Explanation WITH costs
Pink Floyd | Likes : 43045
Dancing With The Stars | Likes : 5379
The Ellen DeGeneres Show | Likes : 16944
American Idol | Likes : 15792
*************************************************
ID:  108
Explanation WITHOUT costs
Six Degrees Of Separation - The Experiment | Likes : 1275
They're, Their, and There have 3 distinct meanings. Learn Them. | Likes : 3842
---
Explanation WITH costs
Star Trek | Likes : 11683
Turn Facebook Pink For 1 Week For Breast Cancer Awareness | Likes : 12942
*